In [1]:
%pwd

'f:\\Git Projects\\Venkey-Chicken-Disease-Classification\\research'

In [2]:
import os
os.chdir("../")
%pwd

'f:\\Git Projects\\Venkey-Chicken-Disease-Classification'

In [3]:
import tensorflow as tf

In [6]:
model = tf.keras.models.load_model("artifacts/training/model.keras", compile=False)
print("Model loaded (compile=False).")

Model loaded (compile=False).


In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [8]:
from ChickenDiseaseClassifier.constants import *
from ChickenDiseaseClassifier.utils.common import read_yaml, create_directories, save_json

In [9]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_validation_config(self) -> EvaluationConfig:
        # Align with training outputs and data dir
        return EvaluationConfig(
            path_of_model=Path("artifacts/training/model.keras"),
            training_data=Path("artifacts/data_ingestion/Chicken-fecal-images"),
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )

In [10]:
from urllib.parse import urlparse

In [ ]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        self.model = None
        self.valid_generator = None
        self.score = None

    def _valid_generator(self):
        # Use a validation split; ensure consistency with how you trained
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.30  # Choose a split for evaluation; independent of training split
        )
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        # Load without compile state, then compile explicitly for evaluation
        return tf.keras.models.load_model(path, compile=False)

    def evaluation(self):
        # Load and compile for evaluation
        self.model = self.load_model(self.config.path_of_model)
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),  # or use self.config.all_params.LEARNING_RATE
            loss="categorical_crossentropy",
            metrics=["accuracy"]
        )

        # Build validation generator
        self._valid_generator()

        # Evaluate; use self.model (not a stray global)
        self.score = self.model.evaluate(self.valid_generator, verbose=1)

    def save_score(self):
        scores = {"loss": float(self.score[0]), "accuracy": float(self.score[1])}
        save_json(path=Path("scores.json"), data=scores)
        print("Saved scores.json:", scores)

In [12]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()

    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
    raise e


[2025-08-16 11:07:20,230: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-16 11:07:20,234: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-16 11:07:20,236: INFO: common: created directory at: artifacts]
Found 1733 images belonging to 7 classes.


f:\Git Projects\Venkey-Chicken-Disease-Classification\.venv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


109/109 ━━━━━━━━━━━━━━━━━━━━ 110s 1s/step - accuracy: 0.7963 - loss: 4.0724
[2025-08-16 11:09:11,102: INFO: common: json file saved at: scores.json]
Saved scores.json: {'loss': 4.072422027587891, 'accuracy': 0.7963069677352905}
